In [71]:
import bs4 as bs
import pickle
import requests
import os
import pandas as pd
import datetime as dt
import pandas_datareader.data as web

In [72]:
def save_sp500_ticker():
    resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, "lxml")
    table = soup.find('table', {'class':'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)
    with open('spxTickers.pickle', 'wb') as f:
        pickle.dump(tickers, f)
    
    #print(tickers)
    
    return tickers

In [88]:
def get_data_from_yahoo(reload_sp500=False):
    if reload_sp500:
        tickers = save_sp500_tickers()
    else:
        with open('spxTickers.pickle', 'rb') as f:
            tickers = pickle.load(f)
            
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')
    
    start = dt.datetime(2013, 1, 1)
    end = dt.datetime.now()
    
    ntikers = len(tickers)
    aux = 1
    for ticker in tickers:
        print(aux,"of",ntikers, ticker)
        if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
            df = web.DataReader(ticker, 'iex', start, end)
            df.to_csv('stock_dfs/{}.csv'.format(ticker))
        else:
            print('Already have {}'.format(ticker))
        aux+=1

In [89]:
get_data_from_yahoo()

1 of 505 MMM
5y
2 of 505 ABT
5y
3 of 505 ABBV
5y
4 of 505 ABMD
5y
5 of 505 ACN
5y
6 of 505 ATVI
5y
7 of 505 ADBE
5y
8 of 505 AMD
5y
9 of 505 AAP
5y
10 of 505 AES
5y
11 of 505 AET
5y
12 of 505 AMG
5y
13 of 505 AFL
5y
14 of 505 A
5y
15 of 505 APD
5y
16 of 505 AKAM
5y
17 of 505 ALK
5y
18 of 505 ALB
5y
19 of 505 ARE
5y
20 of 505 ALXN
5y
21 of 505 ALGN
5y
22 of 505 ALLE
5y
23 of 505 AGN
5y
24 of 505 ADS
5y
25 of 505 LNT
5y
26 of 505 ALL
5y
27 of 505 GOOGL
5y
28 of 505 GOOG
5y
29 of 505 MO
5y
30 of 505 AMZN
5y
31 of 505 AEE
5y
32 of 505 AAL
5y
33 of 505 AEP
5y
34 of 505 AXP
5y
35 of 505 AIG
5y
36 of 505 AMT
5y
37 of 505 AWK
5y
38 of 505 AMP
5y
39 of 505 ABC
5y
40 of 505 AME
5y
41 of 505 AMGN
5y
42 of 505 APH
5y
43 of 505 APC
5y
44 of 505 ADI
5y
45 of 505 ANDV
5y
46 of 505 ANSS
5y
47 of 505 ANTM
5y
48 of 505 AON
5y
49 of 505 AOS
5y
50 of 505 APA
5y
51 of 505 AIV
5y
52 of 505 AAPL
5y
53 of 505 AMAT
5y
54 of 505 APTV
5y
55 of 505 ADM
5y
56 of 505 ARNC
5y
57 of 505 AJG
5y
58 of 505 AIZ
5y
59 of 

459 of 505 UAA
5y
460 of 505 UA
5y
461 of 505 UNP
5y
462 of 505 UAL
5y
463 of 505 UNH
5y
464 of 505 UPS
5y
465 of 505 URI
5y
466 of 505 UTX
5y
467 of 505 UHS
5y
468 of 505 UNM
5y
469 of 505 VFC
5y
470 of 505 VLO
5y
471 of 505 VAR
5y
472 of 505 VTR
5y
473 of 505 VRSN
5y
474 of 505 VRSK
5y
475 of 505 VZ
5y
476 of 505 VRTX
5y
477 of 505 VIAB
5y
478 of 505 V
5y
479 of 505 VNO
5y
480 of 505 VMC
5y
481 of 505 WMT
5y
482 of 505 WBA
5y
483 of 505 DIS
5y
484 of 505 WM
5y
485 of 505 WAT
5y
486 of 505 WEC
5y
487 of 505 WFC
5y
488 of 505 WELL
5y
489 of 505 WDC
5y
490 of 505 WU
5y
491 of 505 WRK
5y
492 of 505 WY
5y
493 of 505 WHR
5y
494 of 505 WMB
5y
495 of 505 WLTW
5y
496 of 505 WYNN
5y
497 of 505 XEL
5y
498 of 505 XRX
5y
499 of 505 XLNX
5y
500 of 505 XL
5y
501 of 505 XYL
5y
502 of 505 YUM
5y
503 of 505 ZBH
5y
504 of 505 ZION
5y
505 of 505 ZTS
5y


In [81]:
start = dt.datetime(2010, 1, 1)
end = dt.datetime.now()
web.DataReader("TSLA", 'robinhood', start, end)

close_price  high_price  interpolated   low_price  \
symbol begins_at                                                      
TSLA   2017-08-01  319.570000  324.450000         False  316.130000   
       2017-08-02  325.890000  327.120000         False  311.220000   
       2017-08-03  347.090000  350.000000         False  343.150000   
       2017-08-04  356.910000  357.270000         False  343.300000   
       2017-08-07  355.170000  359.480000         False  352.750000   
       2017-08-08  365.220000  368.580000         False  357.400000   
       2017-08-09  363.530000  370.000000         False  358.950000   
       2017-08-10  355.400000  366.650400         False  354.660000   
       2017-08-11  357.870000  361.260000         False  353.620000   
       2017-08-14  363.800000  367.660000         False  362.600000   
       2017-08-15  362.330000  365.490000         False  359.370000   
       2017-08-16  362.910000  366.500000         False  362.520000   
       2017-08-17  351.920000  363.300000         False  351.590000   
       2017-08-18  347.460000  354.000000         False  345.800000   
       2017-08-21  337.860000  345.820000         False  331.850000   
       2017-08-22  341.350000  342.240000         False  337.372500   
       2017-08-23  352.770000  353.490000         False  338.304100   
       2017-08-24  352.930000  356.660000         False  349.740000   
       2017-08-25  348.050000  355.690000         False  347.300000   
       2017-08-28  345.660000  347.350000         False  339.720000   
       2017-08-29  347.360000  349.050000         False  338.750000   
       2017-08-30  353.180000  353.470000         False  347.000000   
       2017-08-31  355.900000  358.440000         False  352.820000   
       2017-09-01  355.400000  357.590000         False  353.690200   
       2017-09-05  349.590000  355.490000         False  345.890000   
       2017-09-06  344.530000  350.979000         False  341.560000   
       2017-09-07  350.610000  352.480000         False  343.450000   
       2017-09-08  343.400000  349.780000         False  342.300000   
       2017-09-11  363.690000  363.710000         False  350.000000   
       2017-09-12  362.750000  368.760000         False  360.400000   
...                       ...         ...           ...         ...   
       2018-06-19  352.550000  370.000000         False  346.250000   
       2018-06-20  362.220000  364.380000         False  352.000000   
       2018-06-21  347.510000  366.213900         False  346.270000   
       2018-06-22  333.630000  352.250000         False  332.000000   
       2018-06-25  333.010000  338.470000         False  327.500000   
       2018-06-26  342.000000  343.550000         False  325.799000   
       2018-06-27  344.500000  350.790000         False  339.500000   
       2018-06-28  349.930000  357.020000         False  346.110000   
       2018-06-29  342.950000  353.860000         False  342.410000   
       2018-07-02  335.070000  364.780000         False  329.850000   
       2018-07-03  310.860000  332.490000         False  309.690000   
       2018-07-05  309.160000  314.390000         False  296.220000   
       2018-07-06  308.900000  312.070000         False  302.000000   
       2018-07-09  318.510000  318.520000         False  308.000000   
       2018-07-10  322.470000  327.677100         False  319.200000   
       2018-07-11  318.960000  321.940000         False  315.070000   
       2018-07-12  316.710000  323.230000         False  312.770000   
       2018-07-13  318.870000  319.584900         False  309.250000   
       2018-07-16  310.100000  315.160000         False  306.250000   
       2018-07-17  322.690000  324.740000         False  308.500000   
       2018-07-18  323.850000  325.500000         False  316.250000   
       2018-07-19  320.230000  323.540000         False  314.010000   
       2018-07-20  313.580000  323.240000         False  311.709300   
       2018-07-23  303.200000  305.500000 